<a href="https://colab.research.google.com/github/asiabak/Licencjat-modele/blob/main/GRU_by_Claude_with_ftvecotrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [4]:
!pip install tensorflow

In [6]:
!pip install --upgrade numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 74.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.4 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requ

In [ ]:
!pip install pandas==2.2.2
import os
os.kill(os.getpid(), 9)

In [7]:
# The initial FastText model loading and text preprocessing remains the same
# First, verify GPU is available in Colab
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

# Install necessary libraries if not already available
# !pip install gensim scikit-learn nltk pandas

import urllib.request
import pandas as pd
import re
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from gensim.models import FastText
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, Input, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time
import os

# Use GPU for tensor operations
with tf.device('/GPU:0'):
    # Download NLTK data if needed
    nltk.download('punkt_tab')

    # Set up paths for KGR10 FastText model
    model_url = "https://huggingface.co/clarin-pl/fasttext-kgr10/resolve/main/kgr10.plain.skipgram.dim100.neg10.bin"
    model_path = "kgr10.plain.skipgram.dim100.neg10.bin"

    # Download model if it doesn't exist
    if not os.path.exists(model_path):
        print(f"Downloading KGR10 FastText model from {model_url}...")
        urllib.request.urlretrieve(model_url, model_path)
        print("Download complete!")
    else:
        print(f"Using existing model at {model_path}")

    # Load the FastText model
    print("Loading KGR10 FastText model...")
    model = FastText.load_fasttext_format(model_path)
    print(f"Model loaded! Vector size: {model.vector_size}")

    # Load datasets
    print("Loading datasets...")
    reviews = pd.read_csv("filmweb_jednolity_sentyment.csv")
    reviews_with_idioms = pd.read_csv('filmweb_i_idiomy.csv')
    print(f"Loaded {len(reviews)} reviews and {len(reviews_with_idioms)} reviews with idioms")

    # Text preprocessing
    def preprocess_text(text):
        # Handle NaN values
        if isinstance(text, float) and np.isnan(text):
            return ""

        text = str(text).lower()
        text = re.sub(r'[^\w\s]', '', text)
        words = text.split()
        return ' '.join(words)

    # Apply text preprocessing
    print("Preprocessing text...")
    reviews['review_processed'] = reviews['review'].apply(preprocess_text)
    reviews_with_idioms['review_processed'] = reviews_with_idioms['review'].apply(preprocess_text)

    # Modified vectorization for GRU: We need sequences of word vectors instead of averaging
    def create_sequence_vectors(texts, model, max_length=100):
        # Get sequence of vectors for each text
        def get_sequence_embeddings(text):
            if not text:  # Handle empty strings
                return [np.zeros(model.vector_size)]

            words = word_tokenize(text.lower())
            word_vectors = []
            for word in words[:max_length]:  # Limit sequence length
                try:
                    # Get vector for the word
                    word_vectors.append(model.wv[word])
                except KeyError:
                    # Skip words not in vocabulary
                    continue

            if not word_vectors:
                return [np.zeros(model.vector_size)]
            return word_vectors

        # Process each text in the batch
        sequences = [get_sequence_embeddings(text) for text in texts]

        # Pad sequences to max_length
        padded_sequences = pad_sequences(
            [seq for seq in sequences],
            maxlen=max_length,
            padding='post',
            dtype='float32',
            value=0.0
        )

        return padded_sequences

    # Process reviews in batches to utilize GPU efficiently
    def process_in_batches(df, batch_size=128, max_length=100):
        all_sequences = []
        start_time = time.time()

        for i in range(0, len(df), batch_size):
            batch_texts = df['review_processed'].iloc[i:i+batch_size].values
            batch_sequences = create_sequence_vectors(batch_texts, model, max_length)
            all_sequences.append(batch_sequences)

            # Print progress
            if (i+batch_size) % 1000 == 0 or i+batch_size >= len(df):
                elapsed = time.time() - start_time
                print(f"Processed {i+len(batch_texts)}/{len(df)} reviews in {elapsed:.2f} seconds")

        return np.vstack(all_sequences)

    # Create sequence vectors with GPU acceleration
    MAX_SEQ_LENGTH = 100  # Maximum sequence length
    print("Creating sequence vectors using GPU...")
    reviews_sequences = process_in_batches(reviews, max_length=MAX_SEQ_LENGTH)
    reviews_with_idioms_sequences = process_in_batches(reviews_with_idioms, max_length=MAX_SEQ_LENGTH)

# Prepare data for training
print("Preparing data for GRU training...")
X = reviews_sequences
y = reviews['sentiment'].values
X2 = reviews_with_idioms_sequences
y2 = reviews_with_idioms['sentiment'].values

# Convert labels to categorical for neural network
from tensorflow.keras.utils import to_categorical
y_cat = to_categorical(y, num_classes=3)  # Assuming 3 sentiment classes (0, 1, 2)
y2_cat = to_categorical(y2, num_classes=3)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.4, random_state=12)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2_cat, test_size=0.4, random_state=12)

# Build GRU model
def build_gru_model(input_shape, output_shape):
    model = Sequential()
    model.add(GRU(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64))
    model.add(Dropout(0.2))
    model.add(Dense(output_shape, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

# Train model on reviews only
print("Training GRU on reviews dataset...")
input_shape = (X_train.shape[1], X_train.shape[2])  # (sequence_length, embedding_dim)
output_shape = y_train.shape[1]  # Number of sentiment classes

start_time = time.time()
gru_model = build_gru_model(input_shape, output_shape)
history = gru_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=2,
    batch_size=32,
    verbose=1
)
print(f"GRU training completed in {time.time() - start_time:.2f} seconds")

# Evaluate model
y_pred = np.argmax(gru_model.predict(X_test), axis=1)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Accuracy for GRU model trained on reviews only: {accuracy:.4f}")

# Train model on reviews + idioms
print("Training GRU on reviews + idioms dataset...")
input_shape2 = (X2_train.shape[1], X2_train.shape[2])
output_shape2 = y2_train.shape[1]

start_time = time.time()
gru_model_idioms = build_gru_model(input_shape2, output_shape2)
history2 = gru_model_idioms.fit(
    X2_train, y2_train,
    validation_split=0.2,
    epochs=2,
    batch_size=32,
    verbose=1
)
print(f"GRU training completed in {time.time() - start_time:.2f} seconds")

# Evaluate model
y2_pred = np.argmax(gru_model_idioms.predict(X_test), axis=1)
accuracy2 = accuracy_score(y_test_labels, y2_pred)
print(f"Accuracy for GRU model trained on reviews + idioms: {accuracy2:.4f}")

# Compare model performance
print("\nModel Performance Comparison:")
print(f"Reviews Only:        {accuracy:.4f}")
print(f"Reviews with Idioms: {accuracy2:.4f}")
print(f"Improvement:         {(accuracy2-accuracy)*100:.2f}%")

# Save models if needed
gru_model.save('gru_model_reviews_kgr10.h5')
gru_model_idioms.save('gru_model_reviews_idioms_kgr10.h5')

Training GRU on reviews dataset...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.4356 - loss: 1.0508 - val_accuracy: 0.5203 - val_loss: 1.0256
Epoch 2/2
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4991 - loss: 1.0023 - val_accuracy: 0.5075 - val_loss: 1.0160
GRU training completed in 6.37 seconds
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Accuracy for GRU model trained on reviews only: 0.4718
Training GRU on reviews + idioms dataset...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.4345 - loss: 1.0582 - val_accuracy: 0.4745 - val_loss: 1.0206
Epoch 2/2
66/66 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.5172 - loss: 1.0069 - val_accuracy: 0.4650 - val_loss: 1.0246
GRU training completed in 7.06 seconds
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


Accuracy for GRU model trained on reviews + idioms: 0.4840

Model Performance Comparison:
Reviews Only:        0.4718
Reviews with Idioms: 0.4840
Improvement:         1.22%
